# Install the Required libraries

In [ ]:
%pip install langchain
%pip install langchain_openai
%pip install langchain_community
%pip install pymongo
%pip install python-dotenv
%pip install azure-core
%pip install azure-cosmos
%pip install jq

In [ ]:
%pip install -U gradio
%pip install ipywidgets

# Load the environment variables 

In [1]:
# load the environment variables .env file
import os

from dotenv import load_dotenv

load_dotenv()

True

# Initialize the OpenAI client

In [2]:
import openai

openai.api_type = os.getenv("OPENAI_API_TYPE", "azure")
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT", "https://<YOUR-OPENAI-DEPLOYMENT-NAME>.openai.azure.com/")
openai.api_version = os.getenv("OPENAI_API_VERSION", "2023-09-15-preview")
openai.api_key = os.getenv("OPENAI_API_KEY", "<YOUR-DEPLOYMENT-KEY>")

# Create Database and Collection

In [3]:
from pymongo import MongoClient

# Connection string
mongo_connection_string = os.getenv("AZURE_COSMOS_CONNECTION_STRING", "<YOUR-COSMOS-DB-CONNECTION-STRING>")
mongo_client = MongoClient(mongo_connection_string)

# Database name
db_name = os.getenv("AZURE_COSMOS_DATABASE_NAME", "ResturantChain")
db = mongo_client[db_name]

# Drop the database if it already exists (consider if this is really needed, as it will remove all existing data)
#mongo_client.drop_database(DATABASE_NAME)

# Collection names
collection_names = ["Contoso", "Contoso West", "Users"]

# Iterate through the collection names and create them if they do not exist
for collection_name in collection_names:
    if collection_name not in db.list_collection_names():
        # Creates a collection
        db.create_collection(collection_name)
        print(f"Created collection '{collection_name}'.")
    else:
        print(f"Using existing collection: '{collection_name}'.")

/tmp/ipykernel_51544/3751020075.py:5: UserWarning: You appear to be connected to a CosmosDB cluster. For more information regarding feature compatibility and support please visit https://www.mongodb.com/supportability/cosmosdb
  mongo_client = MongoClient(mongo_connection_string)


Using existing collection: 'Contoso'.
Using existing collection: 'Contoso West'.
Using existing collection: 'Users'.


In [27]:
from langchain.document_loaders.json_loader import JSONLoader

SOURCE_FILE_NAME = "./data/results.json"

loader = JSONLoader(file_path=SOURCE_FILE_NAME, jq_schema=".[].description", text_content=False)
json_data = loader.load_and_split()

In [28]:
print(json_data[1])

page_content='Berries n kale, strawberries, bananas, blueberries kale, tropical fruit blend, and dragon fruit. Our fruity tasty smoothies are blended to perfection.' metadata={'source': '/workspaces/build-24-langchain-vcore/data/results.json', 'seq_num': 2}


In [29]:
from langchain_openai import AzureOpenAIEmbeddings

openai_embeddings_model = os.getenv("AZURE_OPENAI_EMBEDDINGS_MODEL_NAME")
openai_embeddings_deployment = os.getenv("AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT_NAME")

azure_openai_embeddings: AzureOpenAIEmbeddings = AzureOpenAIEmbeddings(
    model=openai_embeddings_model,
    azure_deployment=openai_embeddings_deployment,
)

In [30]:
from langchain.vectorstores.azure_cosmos_db import AzureCosmosDBVectorSearch

collection = db["FoodCollection"]
index_name = os.getenv("AZURE_COSMOS_INDEX_NAME")

vector_store: AzureCosmosDBVectorSearch = AzureCosmosDBVectorSearch.from_documents(
    json_data[0:100],
    azure_openai_embeddings,
    collection=collection,
    index_name=index_name,
)

In [31]:
from langchain_community.vectorstores.azure_cosmos_db import (
    CosmosDBSimilarityType,
    CosmosDBVectorSearchType,
)

# Read more about these variables in detail here. https://learn.microsoft.com/en-us/azure/cosmos-db/mongodb/vcore/vector-search
num_lists = 100
dimensions = 1536
similarity_algorithm = CosmosDBSimilarityType.COS
kind = CosmosDBVectorSearchType.VECTOR_IVF
m = 16
ef_construction = 64
ef_search = 40
score_threshold = 0.1

vector_store.create_index(
    num_lists, dimensions, similarity_algorithm, kind, m, ef_construction
)

{'raw': {'defaultShard': {'numIndexesBefore': 1,
   'numIndexesAfter': 2,
   'createdCollectionAutomatically': False,
   'ok': 1}},
 'ok': 1}

In [40]:
query = "Beef Bacon"
docs = vector_store.similarity_search(query)
print(docs[0].page_content)

Prime beef blend, bacon onion compote, gorgonzola, gruyere, and arugula. With chips or eggplant fries.


In [41]:
from langchain_openai import AzureChatOpenAI

openai_chat_model = os.getenv("AZURE_OPENAI_CHAT_MODEL_NAME")
openai_chat_deployment= os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")

azure_openai_chat: AzureChatOpenAI = AzureChatOpenAI(
    model=openai_chat_model,
    azure_deployment=openai_chat_deployment,
)

In [42]:
# Test the chat
chat_response = azure_openai_chat.invoke("Tell me a joke")
print(chat_response.content)

Why did the tomato turn red?

Because it saw the salad dressing!


In [34]:
order_prompt="""
Hello! You can ask me for food recommendations or customize your order according to your preferences.
{chat_history},
Question: {question}
If you don't know what you want yet, feel free to ask for recommendations!
"""

In [23]:
from langchain.prompts import PromptTemplate

ORDER_PROMPT_TEMPLATE = PromptTemplate.from_template(order_prompt)

In [49]:
from langchain.chains import ConversationalRetrievalChain

retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k":1})

rag_chain = ConversationalRetrievalChain.from_llm(
    llm=azure_openai_chat,
    retriever=retriever,
    condense_question_prompt=ORDER_PROMPT_TEMPLATE,
    return_source_documents=False,
    verbose=False
)

In [111]:
question = "What is beef bacon?"
chat_history = []
response = rag_chain.invoke({"question": question, "chat_history": chat_history})
print(response['answer'])

Beef bacon is a type of bacon that is made from beef instead of pork. It is usually made from beef belly, and is cured and smoked in a similar way to pork bacon.


In [112]:
chat_history.append((question, response['answer']))
question = "Can you tell me more about it?"
response = rag_chain.invoke({"question": question, "chat_history": chat_history})

In [113]:
print(response['answer'])

What are some dishes that use beef bacon?


In [114]:
import gradio as gr


def setup_gradio_interface(chain):    
    with gr.Blocks() as demo_interface:
        chatbot = gr.Chatbot(label="Food Ordering System")
        chat_history = gr.State([])
        lc_chat_history = gr.State([])
        msg = gr.Textbox(label="Your question")
        gr.ClearButton([msg, chatbot])
  
        def fetch_response(message, chat_history, lc_chat_history):
            response = chain.invoke({"question": message, "chat_history": lc_chat_history})
            lc_chat_history.append((message, response['answer']))
            chat_history.append([message, response["answer"]])
            return "", chat_history, lc_chat_history

        msg.submit(fetch_response, inputs=[msg, chatbot, lc_chat_history], outputs=[msg, chatbot, lc_chat_history])
    
    return demo_interface

In [115]:
food_ordering_demo = setup_gradio_interface(rag_chain)
food_ordering_demo.launch()

Running on local URL:  http://127.0.0.1:7900

To create a public link, set `share=True` in `launch()`.


Bad pipe message: %s [b'\nUser-Agent: Mozilla/5.0 (Windows NT 10.0; Win6', b' x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome', b'24.0.0.0 Safari/537.36 Edg/124.0.0.0\r\nAccept-E', b'oding: gzip, deflate, br, zstd\r\nAccept-Language: en-US, en; q=0.9\r\nCookie: _gid=GA1.2.1090558647.1715348779; ', b'a_R1FN4KJKJH=GS1.1.1715353577.2.1.1715357100.0.0.0\r\nOrigin: https://localhost:7900\r\nReferer: h', b'ps://zany-space-funicular-rwgwpqp4qx7h5x7p-7900.app.github.dev/\r\nX-Request-ID: 1681b918647424809f6bb35387a5980b\r\nX-']
Bad pipe message: %s [b'al-IP: 10.240.2.100\r\nX-Forwarded-Port: 443\r\nX-Forwarded-Scheme: https\r\nX-Original', b'RI: /assets/index-D6iiusuW.js\r\nX-Scheme: htt', b'\r\nsec-ch-ua: "Chromium";v="124", "Microsoft Edge";v="124", "Not-A.Brand";v="99"\r\nsec-ch-ua-mobile: ?0\r\nsec-ch-u', b'platform: "Windows"\r\nsec-fetch-site: same-origin\r\nsec-fetch-mode: cors\r\nsec-fetch-dest: script\r\n', b'iority: u=1\r\nx-original-proto: https\r\nX-Forwarded-Proto: https\r\nX-F